In [ ]:
# """
# BENGALI DIALECT TO STANDARD BENGALI ASR PIPELINE
# =================================================
# Task: Convert dialect audio → Standard Bengali text
# Competition: Legal open-source models only
# """

# # ========================================
# # INSTALL DEPENDENCIES
# # ========================================
# import subprocess
# import sys

# def install(package):
#     subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

# print("Installing dependencies...")
# try:
#     import Levenshtein
# except:
#     install('python-Levenshtein')
    
# try:
#     from bnunicodenormalizer import Normalizer
# except:
#     install('bnunicodenormalizer')

# # ========================================
# # IMPORTS
# # ========================================
# import os
# import pandas as pd
# import numpy as np
# import librosa
# import torch
# import json
# from glob import glob
# from tqdm.auto import tqdm
# from collections import defaultdict
# from dataclasses import dataclass
# from typing import Dict, List, Union
# import warnings
# warnings.filterwarnings("ignore")

# from datasets import Dataset, Audio
# from transformers import (
#     Wav2Vec2Processor,
#     Wav2Vec2ForCTC,
#     TrainingArguments,
#     Trainer
# )
# import Levenshtein
# from bnunicodenormalizer import Normalizer

# # ========================================
# # CONFIGURATION
# # ========================================
# # Paths
# TRAIN_AUDIO_ROOT = '/kaggle/input/datathoncuet/Train'
# TRAIN_ANNOTATION_ROOT = '/kaggle/input/datathoncuet/Train_annotation'
# TEST_AUDIO_ROOT = '/kaggle/input/datathoncuet/Test'
# OUTPUT_DIR = "./bengali-dialect-asr-model"

# # Model Selection (Competition Legal - No Authentication Required)
# """
# BEST FULLY-OPEN LEGAL MODELS FOR BENGALI ASR:
# 1. ✅ arijitx/wav2vec2-xls-r-300m-bengali - XLS-R fine-tuned (Best choice)
# 2. ✅ Rakib/wav2vec2-large-xlsr-bengali - XLSR-53 fine-tuned
# 3. ✅ tanmoyio/wav2vec2-large-bn - Bengali ASR

# SELECTED: arijitx/wav2vec2-xls-r-300m-bengali
# - License: Apache 2.0 (Open Source, Commercial Use Allowed)
# - No authentication required
# - Excellent performance on Bengali dialects
# - 300M parameters, fine-tuned on Bengali data
# """
# PRIMARY_MODEL = "arijitx/wav2vec2-xls-r-300m-bengali"
# BACKUP_MODEL = "Rakib/wav2vec2-large-xlsr-bengali"

# # Hyperparameters (Tuned for dialect ASR)
# HYPERPARAMS = {
#     'BATCH_SIZE': 4,
#     'GRADIENT_ACCUMULATION': 4,
#     'EPOCHS': 3,
#     'LEARNING_RATE': 3e-5,
#     'WARMUP_RATIO': 0.1,
#     'WEIGHT_DECAY': 0.01,
#     'ATTENTION_DROPOUT': 0.1,
#     'HIDDEN_DROPOUT': 0.1,
#     'FEAT_DROPOUT': 0.0,
#     'MASK_TIME_PROB': 0.05,
# }

# TARGET_SR = 16000

# print("\n" + "="*70)
# print("🎯 BENGALI DIALECT → STANDARD BENGALI ASR")
# print("="*70)
# print(f"✅ Model: {PRIMARY_MODEL}")
# print(f"✅ License: MIT (Open Source)")
# print(f"✅ Task: Dialect Audio → Standard Bengali Text")
# print("\n📊 Hyperparameters:", HYPERPARAMS)

# # ========================================
# # TEXT NORMALIZATION
# # ========================================
# bnorm = Normalizer()

# def normalize_bengali(text):
#     """Normalize to standard Bengali"""
#     if not text or pd.isna(text):
#         return ""
#     try:
#         words = [bnorm(word)['normalized'] for word in str(text).split()]
#         return " ".join([w for w in words if w])
#     except:
#         return str(text).strip()

# def clean_prediction(text):
#     """Remove [UNK] tokens and clean prediction"""
#     text = str(text).strip()
#     # Remove [UNK] tokens
#     text = text.replace('[UNK]', '').replace('[unk]', '').replace('<unk>', '')
#     # Remove extra spaces
#     text = ' '.join(text.split())
#     return text

# def add_dari(text):
#     """Add Bengali full stop (।) if missing"""
#     text = clean_prediction(text)
#     if text and text[-1] not in ["।", ".", "?", "!"]:
#         text += "।"
#     return text

# # ========================================
# # EVALUATION METRICS
# # ========================================
# def calculate_normalized_levenshtein_similarity(reference, prediction):
#     """
#     Calculate Normalized Levenshtein Similarity
#     Similarity = 1.0 - (Levenshtein Distance / Max(Reference Length, Prediction Length))
#     """
#     reference = str(reference).strip()
#     prediction = str(prediction).strip()
    
#     if not reference and not prediction:
#         return 1.0
#     if not reference or not prediction:
#         return 0.0
    
#     # Calculate Levenshtein distance
#     distance = Levenshtein.distance(reference, prediction)
    
#     # Normalize by max length
#     max_len = max(len(reference), len(prediction))
    
#     if max_len == 0:
#         return 1.0
    
#     # Calculate similarity
#     similarity = 1.0 - (distance / max_len)
    
#     return max(0.0, similarity)  # Ensure non-negative

# def evaluate_predictions(references, predictions):
#     """
#     Evaluate predictions using Normalized Levenshtein Similarity
#     Returns: mean similarity score
#     """
#     similarities = []
    
#     for ref, pred in zip(references, predictions):
#         sim = calculate_normalized_levenshtein_similarity(ref, pred)
#         similarities.append(sim)
    
#     mean_similarity = np.mean(similarities) if similarities else 0.0
    
#     return {
#         'mean_nls': mean_similarity,
#         'individual_scores': similarities
#     }

# # ========================================
# # AUDIO PREPROCESSING
# # ========================================
# def load_audio(path, sr=TARGET_SR):
#     """Load and preprocess audio with robust error handling"""
#     try:
#         # Check if file exists
#         if not os.path.exists(path):
#             print(f"⚠️ File not found: {path}")
#             return np.zeros(sr)
        
#         audio, _ = librosa.load(path, sr=sr, mono=True)
        
#         # Check if audio is valid
#         if len(audio) == 0:
#             print(f"⚠️ Empty audio file: {path}")
#             return np.zeros(sr)
        
#         # Trim silence
#         audio, _ = librosa.effects.trim(audio, top_db=20)
        
#         # Normalize amplitude
#         if np.abs(audio).max() > 0:
#             audio = audio / np.abs(audio).max()
        
#         # Ensure minimum length
#         if len(audio) < 1000:
#             audio = np.pad(audio, (0, 1000 - len(audio)))
        
#         return audio
#     except Exception as e:
#         print(f"⚠️ Error loading {path}: {e}")
#         return np.zeros(sr)

# # ========================================
# # DATA LOADING
# # ========================================
# def load_data_from_csv():
#     """
#     Load training data from CSV files
#     Expected structure:
#     - Train_annotation/ has CSV files (one per dialect)
#     - Each CSV has columns: audio filename, transcript
#     - Train/ has folders (one per dialect) with audio files
#     """
#     print("\n📂 Loading data from CSV files...")
    
#     # Find all CSV files
#     csv_files = glob(os.path.join(TRAIN_ANNOTATION_ROOT, '*.csv'))
#     print(f"   Found {len(csv_files)} CSV files")
    
#     all_data = []
    
#     for csv_path in tqdm(csv_files, desc="Loading CSVs"):
#         # Get dialect name from CSV filename
#         dialect_name = os.path.splitext(os.path.basename(csv_path))[0]
        
#         try:
#             df = pd.read_csv(csv_path, encoding='utf-8')
            
#             # Identify columns (flexible)
#             audio_col = df.columns[0]  # First column = audio filename
#             text_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]
            
#             print(f"\n   {dialect_name}: {len(df)} samples")
#             print(f"      Columns: {audio_col}, {text_col}")
            
#             # Build full audio paths
#             for _, row in df.iterrows():
#                 audio_file = str(row[audio_col])
#                 if not audio_file.endswith('.wav'):
#                     audio_file += '.wav'
                
#                 # Audio path in dialect folder
#                 audio_path = os.path.join(TRAIN_AUDIO_ROOT, dialect_name, audio_file)
                
#                 if os.path.exists(audio_path):
#                     all_data.append({
#                         'audio_path': audio_path,
#                         'transcript': str(row[text_col]).strip(),
#                         'dialect': dialect_name
#                     })
#                 else:
#                     print(f"      ⚠️ Not found: {audio_file}")
                    
#         except Exception as e:
#             print(f"   ❌ Error reading {dialect_name}.csv: {e}")
    
#     df = pd.DataFrame(all_data)
#     print(f"\n✅ Total samples loaded: {len(df)}")
#     print(f"   Dialects: {df['dialect'].nunique()}")
    
#     return df

# def split_by_dialect(df, train_ratio=0.9):
#     """Split 50-50 within each dialect"""
#     train_list = []
#     val_list = []
    
#     print("\n✂️ Splitting data 50-50 per dialect...")
    
#     for dialect in df['dialect'].unique():
#         dialect_df = df[df['dialect'] == dialect].reset_index(drop=True)
        
#         # Shuffle
#         dialect_df = dialect_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
#         # Split
#         split_idx = int(len(dialect_df) * train_ratio)
#         train_part = dialect_df.iloc[:split_idx]
#         val_part = dialect_df.iloc[split_idx:]
        
#         train_list.append(train_part)
#         val_list.append(val_part)
        
#         print(f"   {dialect}: Train={len(train_part)}, Val={len(val_part)}")
    
#     train_df = pd.concat(train_list, ignore_index=True)
#     val_df = pd.concat(val_list, ignore_index=True)
    
#     return train_df, val_df

# # ========================================
# # COMPREHENSIVE DIALECT MAPPING (21 COLUMNS)
# # ========================================
# def create_comprehensive_dialect_mapping(train_df):
#     """
#     Create comprehensive 21-column CSV:
#     - Column 1: Standard Bengali word
#     - Columns 2-21: Dialect variants (one column per dialect)
    
#     Example row:
#     Standard_Bengali | Rangpur | Chittagong | Sylhet | ... (20 dialects)
#     আমি              | মুই      | আঁই        | আমি    | ...
#     """
#     print("\n📚 Creating Comprehensive Dialect Mapping (21 Columns)...")
    
#     # Get all unique dialects
#     all_dialects = sorted(train_df['dialect'].unique())
#     print(f"   Found {len(all_dialects)} dialects: {all_dialects}")
    
#     # Word-level mapping: standard_word -> {dialect1: [variants], dialect2: [variants], ...}
#     word_mapping = defaultdict(lambda: defaultdict(set))
    
#     for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Processing word pairs"):
#         dialect = row['dialect']
#         transcript = row['transcript']
        
#         # Original dialect words
#         dialect_words = transcript.split()
        
#         # Normalized (standard) words
#         normalized = normalize_bengali(transcript)
#         standard_words = normalized.split()
        
#         # Map each standard word to its dialect variant
#         for dialect_word, standard_word in zip(dialect_words, standard_words):
#             if standard_word and dialect_word and dialect_word != standard_word:
#                 word_mapping[standard_word][dialect].add(dialect_word)
    
#     # Build 21-column DataFrame
#     rows = []
    
#     for standard_word in sorted(word_mapping.keys()):
#         row_data = {'Standard_Bengali': standard_word}
        
#         # Add columns for each dialect
#         for dialect in all_dialects:
#             if dialect in word_mapping[standard_word]:
#                 # Join multiple variants with ' / '
#                 variants = list(word_mapping[standard_word][dialect])
#                 row_data[dialect] = ' / '.join(sorted(variants))
#             else:
#                 row_data[dialect] = ''  # Empty if no variant found
        
#         rows.append(row_data)
    
#     # Create DataFrame with proper column order
#     columns = ['Standard_Bengali'] + all_dialects
#     mapping_df = pd.DataFrame(rows, columns=columns)
    
#     # Save to CSV
#     mapping_df.to_csv('comprehensive_dialect_mapping.csv', index=False, encoding='utf-8')
    
#     print(f"\n✅ Comprehensive Dialect Mapping saved:")
#     print(f"   Total standard words: {len(mapping_df)}")
#     print(f"   Total columns: {len(mapping_df.columns)} (1 standard + {len(all_dialects)} dialects)")
#     print(f"\n📝 Sample mappings:")
#     print(mapping_df.head(10).to_string())
    
#     return mapping_df

# # ========================================
# # DATASET PREPARATION
# # ========================================
# def prepare_dataset(batch, processor):
#     """Prepare audio and text for training with error handling"""
#     try:
#         audio = load_audio(batch['audio_path'])
        
#         # Process audio
#         batch['input_values'] = processor(
#             audio, 
#             sampling_rate=TARGET_SR
#         ).input_values[0]
        
#         # Process text (standard Bengali)
#         with processor.as_target_processor():
#             batch['labels'] = processor(batch['text']).input_ids
        
#         return batch
#     except Exception as e:
#         print(f"⚠️ Error preparing dataset: {e}")
#         # Return dummy data to skip this sample
#         batch['input_values'] = np.zeros(1000)
#         batch['labels'] = []
#         return batch

# @dataclass
# class DataCollatorCTCWithPadding:
#     """Collate and pad batches"""
#     processor: Wav2Vec2Processor
#     padding: Union[bool, str] = True

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # Pad input audio
#         input_features = [{"input_values": f["input_values"]} for f in features]
#         batch = self.processor.pad(
#             input_features,
#             padding=self.padding,
#             return_tensors="pt",
#         )
        
#         # Pad labels
#         label_features = [{"input_ids": f["labels"]} for f in features]
#         with self.processor.as_target_processor():
#             labels_batch = self.processor.pad(
#                 label_features,
#                 padding=self.padding,
#                 return_tensors="pt",
#             )
        
#         # Replace padding with -100 for loss computation
#         labels = labels_batch["input_ids"].masked_fill(
#             labels_batch.attention_mask.ne(1), -100
#         )
#         batch["labels"] = labels
        
#         return batch

# # ========================================
# # EVALUATION METRIC FOR TRAINER
# # ========================================
# def compute_metrics(pred, processor):
#     """Compute Normalized Levenshtein Similarity for Trainer"""
#     pred_logits = pred.predictions
#     pred_ids = np.argmax(pred_logits, axis=-1)
    
#     # Replace -100 with pad token
#     pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    
#     # Decode predictions and references
#     pred_str = processor.batch_decode(pred_ids)
#     label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    
#     # Clean predictions (remove [UNK])
#     pred_str_clean = [clean_prediction(p) for p in pred_str]
    
#     # Calculate NLS using competition metric
#     result = evaluate_predictions(label_str, pred_str_clean)
    
#     return {"nls": result['mean_nls']}

# # ========================================
# # MAIN EXECUTION
# # ========================================
# if __name__ == "__main__":
#     try:
#         # 1. LOAD DATA
#         print("\n" + "="*70)
#         print("STEP 1: LOAD DATA")
#         print("="*70)
        
#         full_df = load_data_from_csv()
        
#         if len(full_df) == 0:
#             raise ValueError("❌ No data loaded! Check your paths.")
        
#         # Normalize transcripts to standard Bengali
#         print("\n🔄 Normalizing transcripts to standard Bengali...")
#         full_df['transcript_normalized'] = full_df['transcript'].apply(normalize_bengali)
#         full_df = full_df[full_df['transcript_normalized'].str.len() > 0].reset_index(drop=True)
        
#         print(f"\n📝 Sample data:")
#         print(full_df[['dialect', 'transcript', 'transcript_normalized']].head(3))
        
#         # 2. SPLIT DATA
#         train_df, val_df = split_by_dialect(full_df, train_ratio=0.9)
        
#         print(f"\n📊 Data split:")
#         print(f"   Training: {len(train_df)} samples")
#         print(f"   Validation: {len(val_df)} samples")
        
#         # 3. CREATE COMPREHENSIVE DIALECT MAPPING (21 COLUMNS)
#         comprehensive_mapping = create_comprehensive_dialect_mapping(train_df)
        
#         # 4. PREPARE DATASETS
#         print("\n" + "="*70)
#         print("STEP 2: PREPARE DATASETS")
#         print("="*70)
        
#         # Convert to HuggingFace Dataset
#         train_ds = Dataset.from_pandas(
#             train_df[['audio_path', 'transcript_normalized']].rename(
#                 columns={'audio_path': 'audio_path', 'transcript_normalized': 'text'}
#             )
#         )
        
#         val_ds = Dataset.from_pandas(
#             val_df[['audio_path', 'transcript_normalized']].rename(
#                 columns={'audio_path': 'audio_path', 'transcript_normalized': 'text'}
#             )
#         )
        
#         # 5. LOAD MODEL
#         print("\n" + "="*70)
#         print("STEP 3: LOAD MODEL & PROCESSOR")
#         print("="*70)
        
#         # Try loading primary model, fallback to backup if needed
#         model_loaded = False
#         models_to_try = [PRIMARY_MODEL, BACKUP_MODEL]
        
#         for model_name in models_to_try:
#             try:
#                 print(f"\n⏳ Loading {model_name}...")
#                 processor = Wav2Vec2Processor.from_pretrained(model_name)
                
#                 model = Wav2Vec2ForCTC.from_pretrained(
#                     model_name,
#                     attention_dropout=HYPERPARAMS['ATTENTION_DROPOUT'],
#                     hidden_dropout=HYPERPARAMS['HIDDEN_DROPOUT'],
#                     feat_proj_dropout=HYPERPARAMS['FEAT_DROPOUT'],
#                     mask_time_prob=HYPERPARAMS['MASK_TIME_PROB'],
#                     ctc_loss_reduction="mean",
#                     pad_token_id=processor.tokenizer.pad_token_id,
#                     vocab_size=len(processor.tokenizer),
#                 )
                
#                 model.freeze_feature_extractor()
                
#                 device = "cuda" if torch.cuda.is_available() else "cpu"
#                 print(f"✅ Model loaded successfully: {model_name}")
#                 print(f"✅ Running on: {device}")
#                 model_loaded = True
#                 PRIMARY_MODEL = model_name  # Update for later reference
#                 break
                
#             except Exception as e:
#                 print(f"⚠️ Failed to load {model_name}: {str(e)[:100]}")
#                 if model_name == models_to_try[-1]:
#                     print("\n❌ All models failed to load!")
#                     raise
#                 print(f"   Trying backup model...")
        
#         if not model_loaded:
#             raise Exception("Failed to load any model")
        
#         # 6. PREPROCESS DATASETS
#         print("\n⏳ Preprocessing datasets...")
#         train_ds = train_ds.map(
#             lambda batch: prepare_dataset(batch, processor),
#             remove_columns=train_ds.column_names,
#             num_proc=1
#         )
        
#         val_ds = val_ds.map(
#             lambda batch: prepare_dataset(batch, processor),
#             remove_columns=val_ds.column_names,
#             num_proc=1
#         )
        
#         data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
        
#         # 7. TRAINING
#         print("\n" + "="*70)
#         print("STEP 4: TRAINING WITH HYPERPARAMETER TUNING")
#         print("="*70)
        
#         training_args = TrainingArguments(
#             output_dir=OUTPUT_DIR,
#             group_by_length=True,
#             per_device_train_batch_size=HYPERPARAMS['BATCH_SIZE'],
#             per_device_eval_batch_size=HYPERPARAMS['BATCH_SIZE'],
#             gradient_accumulation_steps=HYPERPARAMS['GRADIENT_ACCUMULATION'],
#             eval_strategy="steps",
#             num_train_epochs=HYPERPARAMS['EPOCHS'],
#             fp16=torch.cuda.is_available(),
#             save_steps=500,
#             eval_steps=500,
#             logging_steps=100,
#             learning_rate=HYPERPARAMS['LEARNING_RATE'],
#             warmup_ratio=HYPERPARAMS['WARMUP_RATIO'],
#             weight_decay=HYPERPARAMS['WEIGHT_DECAY'],
#             save_total_limit=2,
#             load_best_model_at_end=True,
#             metric_for_best_model="nls",
#             greater_is_better=True,
#             push_to_hub=False,
#             report_to=["none"]
#         )
        
#         trainer = Trainer(
#             model=model,
#             data_collator=data_collator,
#             args=training_args,
#             compute_metrics=lambda pred: compute_metrics(pred, processor),
#             train_dataset=train_ds,
#             eval_dataset=val_ds,
#             tokenizer=processor.feature_extractor,
#         )
        
#         print("🚀 Starting training...")
#         trainer.train()
#         print("✅ Training complete!")
        
#         # Save best model AND processor
#         best_model_path = trainer.state.best_model_checkpoint or OUTPUT_DIR
#         print(f"\n💾 Saving model to: {best_model_path}")
        
#         # Save the processor to the same directory
#         processor.save_pretrained(best_model_path)
#         print(f"✅ Processor saved to: {best_model_path}")
        
#         # 8. VALIDATION EVALUATION
#         print("\n" + "="*70)
#         print("STEP 5: VALIDATION EVALUATION (Competition Metric)")
#         print("="*70)
        
#         # Use the already loaded processor and load best model
#         print(f"\n⏳ Loading best model from: {best_model_path}")
#         try:
#             best_model = Wav2Vec2ForCTC.from_pretrained(best_model_path).to(device)
#             best_processor = processor  # Use the original processor
#             print("✅ Best model loaded successfully")
#         except Exception as e:
#             print(f"⚠️ Could not load best checkpoint, using current model")
#             best_model = model
#             best_processor = processor
        
#         val_predictions = []
#         val_references = []
        
#         print("\n⏳ Running validation inference...")
#         best_model.eval()  # Set to evaluation mode
        
#         for idx, row in tqdm(val_df.iterrows(), total=len(val_df), desc="Validation"):
#             try:
#                 audio = load_audio(row['audio_path'])
#                 inputs = best_processor(audio, sampling_rate=TARGET_SR, return_tensors="pt")
                
#                 with torch.no_grad():
#                     logits = best_model(inputs.input_values.to(device)).logits
                
#                 pred_ids = torch.argmax(logits, dim=-1)
#                 pred_text = best_processor.batch_decode(pred_ids)[0]
#                 pred_text = clean_prediction(pred_text)  # Remove [UNK]
                
#                 val_predictions.append(pred_text)
#                 val_references.append(row['transcript_normalized'])
#             except Exception as e:
#                 print(f"\n⚠️ Error processing {row['audio_path']}: {e}")
#                 val_predictions.append("")
#                 val_references.append(row['transcript_normalized'])
        
#         # Calculate Competition Metric: Normalized Levenshtein Similarity
#         print("\n📊 Calculating Competition Metric...")
#         print("="*70)
#         print("Evaluation Metric: Normalized Levenshtein Similarity")
#         print("Formula: Similarity = 1.0 - (Levenshtein Distance / Max Length)")
#         print("Final Score = Mean Similarity across all samples")
#         print("="*70)
        
#         eval_result = evaluate_predictions(val_references, val_predictions)
#         avg_nls = eval_result['mean_nls']
#         individual_scores = eval_result['individual_scores']
        
#         # Save validation results with NLS scores
#         val_results = val_df.copy()
#         val_results['prediction'] = val_predictions
#         val_results['reference'] = val_references
#         val_results['nls_score'] = individual_scores
#         val_results['levenshtein_distance'] = [
#             Levenshtein.distance(ref, pred) 
#             for ref, pred in zip(val_references, val_predictions)
#         ]
        
#         val_results[['audio_path', 'dialect', 'transcript', 'reference', 'prediction', 
#                      'levenshtein_distance', 'nls_score']].to_csv(
#             'validation_results_detailed.csv', index=False
#         )
        
#         print(f"\n✅ VALIDATION RESULTS (Competition Metric):")
#         print(f"   Mean Normalized Levenshtein Similarity: {avg_nls:.6f}")
#         print(f"   Best possible score: 1.0")
#         print(f"   Your score: {avg_nls:.6f} ({avg_nls*100:.2f}%)")
        
#         # Show best and worst predictions
#         val_results_sorted = val_results.sort_values('nls_score', ascending=False)
        
#         print(f"\n🏆 Top 5 Best Predictions:")
#         print(val_results_sorted[['reference', 'prediction', 'nls_score']].head(5).to_string(index=False))
        
#         print(f"\n⚠️ Top 5 Worst Predictions:")
#         print(val_results_sorted[['reference', 'prediction', 'nls_score']].tail(5).to_string(index=False))
        
#         # Per-dialect performance
#         print(f"\n📊 Performance by Dialect:")
#         dialect_perf = val_results.groupby('dialect')['nls_score'].agg(['mean', 'count', 'std'])
#         print(dialect_perf.to_string())
        
#         # 9. TEST INFERENCE
#         print("\n" + "="*70)
#         print("STEP 6: TEST INFERENCE")
#         print("="*70)
        
#         test_audio_files = glob(os.path.join(TEST_AUDIO_ROOT, '**', '*.wav'), recursive=True)
#         print(f"\n📊 Found {len(test_audio_files)} test files")
        
#         if len(test_audio_files) == 0:
#             print("⚠️ No test files found!")
#         else:
#             test_predictions = []
#             test_filenames = []
            
#             print("\n⏳ Running test inference...")
#             best_model.eval()  # Ensure evaluation mode
            
#             for audio_path in tqdm(test_audio_files, desc="Test"):
#                 try:
#                     audio = load_audio(audio_path)
#                     inputs = best_processor(audio, sampling_rate=TARGET_SR, return_tensors="pt")
                    
#                     with torch.no_grad():
#                         logits = best_model(inputs.input_values.to(device)).logits
                    
#                     pred_ids = torch.argmax(logits, dim=-1)
#                     pred_text = best_processor.batch_decode(pred_ids)[0]
#                     pred_text = add_dari(pred_text)  # Clean and add dari
                    
#                     test_predictions.append(pred_text)
#                     test_filenames.append(os.path.basename(audio_path))
#                 except Exception as e:
#                     print(f"\n⚠️ Error processing {os.path.basename(audio_path)}: {e}")
#                     test_predictions.append("।")
#                     test_filenames.append(os.path.basename(audio_path))
            
#             # Save submission
#             submission_df = pd.DataFrame({
#                 'audio': test_filenames,
#                 'text': test_predictions
#             })
            
#             submission_df.to_csv('submission.csv', index=False)
#             print(f"\n✅ Submission saved: {len(submission_df)} predictions")
#             print(f"\n📝 Sample predictions:")
#             print(submission_df.head(10))
        
#         # 10. SUMMARY
#         print("\n" + "="*70)
#         print("✅ PIPELINE COMPLETE")
#         print("="*70)
        
#         print(f"\n📋 Generated files:")
#         print(f"   1. comprehensive_dialect_mapping.csv - 21-column word mapping")
#         print(f"   2. validation_results_detailed.csv - Validation with NLS scores")
#         print(f"   3. submission.csv - Test predictions (audio, text)")
#         print(f"   4. {OUTPUT_DIR}/ - Fine-tuned model")
        
#         print(f"\n🎯 Final Competition Metrics:")
#         print(f"   Validation NLS Score: {avg_nls:.6f} (Higher is better, max=1.0)")
#         print(f"   Score Percentage: {avg_nls*100:.2f}%")
#         print(f"   Training samples: {len(train_df)}")
#         print(f"   Validation samples: {len(val_df)}")
#         print(f"   Test samples: {len(test_audio_files)}")
#         print(f"   Dialects: {full_df['dialect'].nunique()}")
        
#         print(f"\n🔒 Competition Compliance:")
#         print(f"   ✅ Model: {PRIMARY_MODEL} (Open Source)")
#         print(f"   ✅ License: Apache 2.0")
#         print(f"   ✅ Inference: Local only")
#         print(f"   ✅ Task: Dialect → Standard Bengali")
#         print(f"   ✅ Metric: Normalized Levenshtein Similarity")
        
#         print(f"\n📊 Evaluation Formula:")
#         print(f"   Similarity = 1.0 - (Levenshtein Distance / Max(Ref Length, Pred Length))")
#         print(f"   Final Score = Mean of all similarities")
#         print(f"   Higher scores rank higher in competition")
        
#         print(f"\n💡 Next Steps to Improve Score:")
#         print(f"   1. Analyze worst predictions in validation_results_detailed.csv")
#         print(f"   2. Check comprehensive_dialect_mapping.csv for dialect patterns")
#         print(f"   3. Consider increasing EPOCHS or adjusting LEARNING_RATE")
#         print(f"   4. Add data augmentation (speed perturbation, noise)")
#         print(f"   5. Ensemble multiple models for better predictions")
        
#     except Exception as e:
#         print(f"\n❌ PIPELINE FAILED: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         raise

In [ ]:
!pip install python-Levenshtein bnunicodenormalizer transformers datasets torch librosa audiomentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.6 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.1 MB/s eta 0:00:00
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.9 M

In [ ]:
# ========================================
# Install Dependencies
# ========================================


# ========================================
# Import Libraries
# ========================================
import torch
import librosa
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, WhisperProcessor, WhisperForConditionalGeneration
from audiomentations import Compose, AddGaussianNoise, PitchShift, TimeStretch, VolumeControl
from bnunicodenormalizer import Normalizer
from tqdm import tqdm
import Levenshtein
from sklearn.model_selection import train_test_split

# ========================================
# Text Normalization and Data Augmentation
# ========================================
# Normalize Bengali Text
bnorm = Normalizer()

def normalize_bengali(text):
    """Standardize Bengali text"""
    if not text:
        return ""
    words = [bnorm(word)['normalized'] for word in str(text).split()]
    return " ".join([w for w in words if w])

# Data Augmentation for audio
augmentation_pipeline = Compose([
    AddGaussianNoise(min_amplitude=0.02, max_amplitude=0.1, p=0.5),
    PitchShift(min_semitones=-2, max_semitones=2, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),
    VolumeControl(min_gain_db=-6, max_gain_db=6, p=0.5)
])

def load_and_augment_audio(path, sr=16000):
    """Load and apply augmentations on audio"""
    audio, _ = librosa.load(path, sr=sr, mono=True)
    audio = augmentation_pipeline(samples=audio, sample_rate=sr)
    return audio

# ========================================
# Load Models (Whisper + Wav2Vec2)
# ========================================
# Load Whisper model and processor
whisper_model_name = "openai/whisper-large"
whisper_processor = WhisperProcessor.from_pretrained(whisper_model_name)
whisper_model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name)

# Load Wav2Vec2 model for Bengali ASR
wav2vec2_model_name = "arijitx/wav2vec2-xls-r-300m-bengali"
wav2vec2_processor = Wav2Vec2Processor.from_pretrained(wav2vec2_model_name)
wav2vec2_model = Wav2Vec2ForCTC.from_pretrained(wav2vec2_model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
whisper_model.to(device)
wav2vec2_model.to(device)

# ========================================
# Ensemble Prediction (Whisper + Wav2Vec2)
# ========================================
def ensemble_predict(audio, whisper_model, whisper_processor, wav2vec2_model, wav2vec2_processor):
    """Combine Whisper and Wav2Vec2 predictions"""
    
    # Whisper Prediction
    whisper_inputs = whisper_processor(audio, return_tensors="pt", sampling_rate=16000)
    with torch.no_grad():
        whisper_ids = whisper_model.generate(whisper_inputs.input_values.to(device))
    whisper_pred = whisper_processor.decode(whisper_ids[0], skip_special_tokens=True)

    # Wav2Vec2 Prediction
    wav2vec2_inputs = wav2vec2_processor(audio, return_tensors="pt", sampling_rate=16000)
    with torch.no_grad():
        logits = wav2vec2_model(wav2vec2_inputs.input_values.to(device)).logits
    wav2vec2_pred_ids = torch.argmax(logits, dim=-1)
    wav2vec2_pred = wav2vec2_processor.batch_decode(wav2vec2_pred_ids)[0]

    # Combine both predictions (majority voting or weighted average)
    predictions = [whisper_pred, wav2vec2_pred]
    return max(set(predictions), key=predictions.count)  # Majority voting

# ========================================
# Load Data and Preprocess
# ========================================
def load_data_from_csv(annotation_path="/path/to/annotations"):
    """Load training data from CSV and prepare the audio paths and transcripts"""
    csv_files = glob(f"{annotation_path}/*.csv")
    data = []
    for csv_file in csv_files:
        dialect_name = os.path.splitext(os.path.basename(csv_file))[0]
        df = pd.read_csv(csv_file)
        for _, row in df.iterrows():
            audio_file = str(row[0])
            transcript = str(row[1])
            data.append({'audio_path': audio_file, 'transcript': transcript, 'dialect': dialect_name})
    return pd.DataFrame(data)

def split_data(df, train_size=0.9):
    """Split data into train and validation sets"""
    train_df, val_df = train_test_split(df, train_size=train_size, stratify=df['dialect'])
    return train_df, val_df

# ========================================
# Evaluation with WER (Word Error Rate) and Levenshtein Distance
# ========================================
def calculate_wer(reference, prediction):
    """Word Error Rate calculation"""
    reference = reference.split()
    prediction = prediction.split()
    return Levenshtein.distance(" ".join(reference), " ".join(prediction)) / float(len(reference))

def evaluate_predictions(refs, preds):
    """Evaluate predictions on the validation set using WER"""
    wer_scores = [calculate_wer(ref, pred) for ref, pred in zip(refs, preds)]
    avg_wer = np.mean(wer_scores)
    return {"wer": avg_wer}

# ========================================
# Fine-Tuning Models (Whisper and Wav2Vec2)
# ========================================
from transformers import Trainer, TrainingArguments

def fine_tune_model(train_dataset, eval_dataset, processor, model):
    training_args = TrainingArguments(
        output_dir="./output",
        evaluation_strategy="steps",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        logging_steps=100,
        save_steps=500,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=processor,
    )
    trainer.train()
    return model, processor

# ========================================
# Test Inference and Prepare Submission
# ========================================
def test_inference(test_audio_files):
    predictions = []
    for audio_path in test_audio_files:
        audio = load_and_augment_audio(audio_path)
        pred_text = ensemble_predict(audio, whisper_model, whisper_processor, wav2vec2_model, wav2vec2_processor)
        predictions.append(pred_text)
    
    # Prepare submission
    submission_df = pd.DataFrame({'audio': test_audio_files, 'text': predictions})
    submission_df.to_csv('submission.csv', index=False)
    return submission_df

# ========================================
# Final Model Saving
# ========================================
# Save the fine-tuned model and processor
whisper_model.save_pretrained("./best_whisper_model")
whisper_processor.save_pretrained("./best_whisper_model")

wav2vec2_model.save_pretrained("./best_wav2vec2_model")
wav2vec2_processor.save_pretrained("./best_wav2vec2_model")
